In [ ]:
import os
# [print(f'{k: <25}\t{v}') for k,v in os.environ.items()];

In [2]:
NOTEBOOK_LOCATION = 'cloud' if os.environ.get('USER','') == 'azureuser' else 'local'
print(NOTEBOOK_LOCATION)

In [3]:
from azureml.core.authentication import InteractiveLoginAuthentication
if NOTEBOOK_LOCATION == 'local':
    TENANT_ID = input('Tenant ID: ')
    SUBSCRIPTION_ID = input('Subscription ID: ')
    RESOURCE_GROUP = input('Resource group: ')
    WORKSPACE_NAME = input('Workspace name: ')
    auth = InteractiveLoginAuthentication(force=True, tenant_id=TENANT_ID)

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
from azureml.core import Workspace

if NOTEBOOK_LOCATION == 'local':
    ws = Workspace.get(
        name=WORKSPACE_NAME,
        subscription_id=SUBSCRIPTION_ID,
        resource_group=RESOURCE_GROUP
    )
else:
    ws = Workspace.from_config()
    
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-121376
Azure region: southcentralus
Subscription id: 30d182b7-c8c4-421c-8fa0-d3037ecfe6d2
Resource group: aml-quickstarts-121376


In [5]:
from azureml.core import Experiment

In [6]:
ws.compute_targets.keys()

dict_keys(['auto-ml', 'notebook121376'])

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "aml-compute")
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 2)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Using existing compute target: ' + compute_name)
else:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        min_nodes=compute_min_nodes,
        max_nodes=compute_max_nodes
    )
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

Creating
Succeeded.......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# HyperDrive

In [63]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import loguniform, choice

hyd_est = SKLearn(
    source_directory='./',
    entry_script='train.py',
    compute_target=compute_target
)
'''
hyperparameters for sklearn.linear_model.LogisticRegression:
penalty='l2'   # ['l1', 'l2', 'elasticnet', 'none']
C=1.0   # [.001, .01, .1, 1, 10, 100, 1000]
solver='lbfgs'   # ['liblinear', 'saga']
max_iter=100   # [125, 150]
multi_class='auto'   # ['multinomial', 'ovr']
'''

# Specify parameter sampler
ps = RandomParameterSampling({
    "penalty": choice('l1', 'l2'),
    "C": loguniform(-4, 3),
    "solver": choice('lbfgs', 'saga')
})

# Specify a Policy for early stopping
policy = BanditPolicy(
    evaluation_interval = 10,
    slack_factor = 0.2
)

In [65]:
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=hyd_est,
    hyperparameter_sampling=ps,
    primary_metric_name='norm_macro_recall',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    max_total_runs=50,
    max_concurrent_runs=4
)

In [11]:
hyd_exp = Experiment(workspace=ws, name="udacity-project")

In [ ]:
compute_target.wait_for_completion(show_output=True)

In [68]:
hyd_run = hyd_exp.submit(config=hyperdrive_config)
hyd_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_097e76a8-9ecc-4308-acff-ce915cd23202,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [76]:
hyd_best_run = hyd_run.get_best_run_by_primary_metric()
hyd_best_run_metrics = hyd_best_run.get_metrics()

print('Best Run Id: ', hyd_best_run.id)
print('Accuracy: ', hyd_best_run_metrics['accuracy'])
print('F1 score: ', hyd_best_run_metrics['AUC_weighted'])

Best Run Id:  HD_5c402c15-2bfc-4560-a34a-ebbed2a234aa_25
Accuracy:  0.9005563985837127
F1 score:  0.340710932260228


In [91]:
# Get your best run and save the model from that run.
hyd_model = hyd_best_run.register_model(
    model_name='bank-marketing-hyperdrive-best-model',
    model_path=os.path.join('outputs','model.joblib')
)

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path bankmarketing-amlproject in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_a74db4c82d09cfa59fe38e5740c4f5fe87c1c4e2d428aa2028ead362d31a34b6_d.txt', 'azureml-logs/65_job_prep-tvmps_a74db4c82d09cfa59fe38e5740c4f5fe87c1c4e2d428aa2028ead362d31a34b6_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_a74db4c82d09cfa59fe38e5740c4f5fe87c1c4e2d428aa2028ead362d31a34b6_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/108_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path bankmarketing-amlproject in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_a

# AutoML

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_file_source = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_file_source)

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)

In [11]:
datastore = ws.get_default_datastore()
type(datastore)

azureml.data.azure_storage_datastore.AzureBlobDatastore

In [12]:
x_train.assign(target=y_train).to_csv('df_train.csv')

In [13]:
datastore.upload_files(['df_train.csv'], overwrite=True)

Uploading an estimated of 1 files
Uploading df_train.csv
Uploaded df_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [14]:
train_ds = TabularDatasetFactory.from_delimited_files((datastore, 'df_train.csv'))

In [15]:
type(train_ds)

azureml.data.tabular_dataset.TabularDataset

In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

aml_classification_metrics = [
    'accuracy',
    'AUC_weighted',
    'average_precision_score_weighted',
    'norm_macro_recall',
    'precision_score_weighted'
]

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='norm_macro_recall',
    training_data=train_ds,
    label_column_name='target',
    compute_target=compute_target,
    n_cross_validations=5
)

In [17]:
compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [18]:
# Submit your automl run

from azureml.core.experiment import Experiment

automl_exp = Experiment(ws, "automl_test_experiment")
automl_run = automl_exp.submit(config=automl_config)
automl_run

Running on remote.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_test_experiment,AutoML_9b2681b7-92ea-4e12-957d-2bcb464540a6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.
automl_best_run = automl_run.get_best_run_by_primary_metric()
automl_best_run_metrics = automl_best_run.get_metrics()

print('Best Run Id: ', automl_best_run.id)
print('Accuracy: ', automl_best_run_metrics['accuracy'])
print('Normalized recall score: ', automl_best_run_metrics['norm_macro_recall'])

model = automl_best_run.register_model(
    model_name='bank-marketing-automl-best-model',
    model_path=os.path.join('outputs','model.pkl')
)

In [27]:
compute_target.delete()
compute_target.wait_for_completion(show_output=True, is_delete_operation=True)

Deleting........
SucceededProvisioning operation finished, operation "Succeeded"
